In [1]:
import os
%pwd

'/home/zeerak14/Projects/iNeuron/MLOPS/Kidney_Disease_Classification_Using_MLOPS/research'

In [2]:
os.chdir("../")
%pwd

'/home/zeerak14/Projects/iNeuron/MLOPS/Kidney_Disease_Classification_Using_MLOPS'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from Kidney_Tumor_Classification.constants import *
from Kidney_Tumor_Classification.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
from zipfile import ZipFile
import gdown
from Kidney_Tumor_Classification import logger
from Kidney_Tumor_Classification.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the url and save it to the local file system.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} to {zip_download_dir}")
            
        except Exception as e:
            raise e

    def extract_zip_file(self)-> None:
        """
        Extracts the zip file into the data directory.

        Returns:
            None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

2023-10-15 11:58:47,247 : INFO : common :common.py : 32: yaml file: config/config.yaml loaded successfully
2023-10-15 11:58:47,301 : INFO : common :common.py : 32: yaml file: params.yaml loaded successfully
2023-10-15 11:58:47,304 : INFO : common :common.py : 52: created directory at: artifacts
2023-10-15 11:58:47,311 : INFO : common :common.py : 52: created directory at: artifacts/data_ingestion
2023-10-15 11:58:47,317 : INFO : 2470129960 :2470129960.py : 13: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip


Downloading...
From (uriginal): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=58da5170-5cd6-4810-afc6-6e6c04f3e552
To: /home/zeerak14/Projects/iNeuron/MLOPS/Kidney_Disease_Classification_Using_MLOPS/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:50<00:00, 1.15MB/s]

2023-10-15 11:59:43,615 : INFO : 2470129960 :2470129960.py : 18: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip


In [10]:
%pwd


'/home/zeerak14/Projects/iNeuron/MLOPS/Kidney_Disease_Classification_Using_MLOPS'